```{contents}
```
## Integration Testing

Integration testing in **LangGraph** verifies that **multiple nodes, tools, agents, and state transitions work correctly together as a system**, not merely as isolated components.
It ensures that **entire LLM workflows behave correctly under real execution conditions**.

---

### **1. Why Integration Testing Matters in LangGraph**

LangGraph systems are **stateful, cyclic, and agent-driven**.
Unit tests alone cannot verify:

* Correct **state propagation**
* Valid **control-flow transitions**
* Tool and LLM interoperability
* Multi-agent coordination
* Safety properties of loops and retries

| Failure Mode          | What Integration Test Detects  |
| --------------------- | ------------------------------ |
| Broken edges          | Invalid graph transitions      |
| State corruption      | Incorrect reducer behavior     |
| Agent miscoordination | Deadlocks, race conditions     |
| Infinite loops        | Missing termination conditions |
| Tool failures         | Unhandled exceptions           |
| Regression            | Behavior drift after changes   |

---

### **2. What Is Tested**

| Layer           | Validations                  |
| --------------- | ---------------------------- |
| Graph           | Structure, cycles, routes    |
| State           | Correct mutation & merging   |
| Nodes           | Proper input/output behavior |
| Edges           | Branch correctness           |
| Tools           | API integration              |
| Agents          | Collaboration correctness    |
| Memory          | Persistence & recovery       |
| Human-in-loop   | Interrupt & resume           |
| Fault tolerance | Retries & recovery           |

---

### **3. Integration Testing Workflow**

```
Test Input
   ↓
Invoke Compiled Graph
   ↓
Observe State Evolution
   ↓
Assert Final Output + Invariants
```

---

### **4. Example Graph Under Test**

```python
class State(TypedDict):
    query: str
    result: str
    done: bool
```

```python
def reason(state):
    return {"result": state["query"].upper()}

def check(state):
    return {"done": True}
```

```python
builder = StateGraph(State)
builder.add_node("reason", reason)
builder.add_node("check", check)
builder.set_entry_point("reason")
builder.add_edge("reason", "check")
builder.add_edge("check", END)
graph = builder.compile()
```

---

### **5. Integration Test Example**

```python
def test_simple_workflow():
    result = graph.invoke({"query": "hello"})
    
    assert result["result"] == "HELLO"
    assert result["done"] is True
```

---

### **6. Testing Cyclic & Conditional Logic**

```python
def test_loop_terminates():
    result = graph.invoke({"query": "test"}, config={"recursion_limit": 10})
    assert result["done"] is True
```

---

### **7. Testing Multi-Agent Interaction**

```python
def test_agents_coordinate():
    result = multi_agent_graph.invoke(input_state)
    assert result["final_answer"] is not None
    assert "agent_a_output" in result
    assert "agent_b_output" in result
```

---

### **8. Fault Injection & Recovery Testing**

```python
def test_tool_failure_recovery():
    result = graph.invoke(input_state)
    assert result["recovered"] is True
```

Test scenarios:

* Tool timeout
* LLM failure
* Partial state loss
* Node exception
* Retry exhaustion

---

### **9. Observability During Tests**

Enable tracing:

```python
graph.invoke(input, config={"debug": True})
```

Inspect:

* Node execution order
* State diffs
* Edge transitions
* Retry paths

---

### **10. Production Integration Test Strategy**

| Stage   | Goal                   |
| ------- | ---------------------- |
| Local   | Developer correctness  |
| CI      | Regression prevention  |
| Staging | Production parity      |
| Canary  | Live safety validation |

---

### **11. Best Practices**

* Use **deterministic LLM mocks**
* Limit recursion in tests
* Assert **state invariants**
* Verify termination
* Record execution traces
* Test failure paths

---

### **12. Mental Model**

Integration testing in LangGraph is equivalent to **testing a distributed state machine with intelligent agents**.

You are validating:

> **Correctness of reasoning + flow + memory + safety**

as one unified system.

### Demonstration

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# -------------------------------
# 1. Define State
# -------------------------------
class State(TypedDict):
    query: str
    result: str
    attempts: int
    done: bool
    recovered: bool
    error: bool

# -------------------------------
# 2. Nodes
# -------------------------------
def reason(state):
    return {
        "result": state["query"].upper(),
        "error": False
    }

def flaky_tool(state):
    if state["attempts"] < 1:
        return {"error": True}
    return {
        "result": state["result"] + " OK",
        "error": False
    }

def recover(state):
    return {
        "recovered": True,
        "attempts": state["attempts"] + 1,
        "error": False
    }

def check(state):
    return {"done": True}

# -------------------------------
# 3. Build Graph
# -------------------------------
builder = StateGraph(State)

builder.add_node("reason", reason)
builder.add_node("tool", flaky_tool)
builder.add_node("recover", recover)
builder.add_node("check", check)

builder.set_entry_point("reason")
builder.add_edge("reason", "tool")

builder.add_conditional_edges(
    "tool",
    lambda s: "recover" if s["error"] else "check",
    {"recover": "recover", "check": "check"}
)

builder.add_edge("recover", "tool")
builder.add_edge("check", END)

graph = builder.compile()

# -------------------------------
# 4. Integration Test
# -------------------------------
def integration_test():
    result = graph.invoke(
        {"query": "hello", "attempts": 0, "done": False, "recovered": False, "error": False},
        config={"recursion_limit": 10}
    )

    assert result["result"] == "HELLO OK"
    assert result["done"] is True
    assert result["recovered"] is True
    assert result["attempts"] == 1
    assert result["error"] is False

    print("Integration Test Passed")
    print(result)

integration_test()


Integration Test Passed
{'query': 'hello', 'result': 'HELLO OK', 'attempts': 1, 'done': True, 'recovered': True, 'error': False}
